## Extract Earth System Science Data (ESSD) datasets from S3 into Pachyderm

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

In [1]:
import os
import boto3
import openpyxl

from osc_ingest_trino import load_credentials_dotenv

import python_pachyderm

import io

from pandas_profiling import ProfileReport

Load Credentials

In [2]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

# Load environment variables from credentials.env
load_credentials_dotenv()

Create an S3 resource for the bucket holding source data

In [3]:
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [4]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [5]:
client.delete_repo("essd")

In [6]:
# Create a new pachyderm repo called `gppd`
client.create_repo("essd")

In [7]:
# Display list of repos
print(list(client.list_repo()))

[repo {
  name: "essd"
  type: "user"
}
created {
  seconds: 1660385394
  nanos: 931518000
}
, repo {
  name: "wri-gppd"
  type: "user"
}
created {
  seconds: 1659790074
  nanos: 399079000
}
size_bytes_upper_bound: 27086020
branches {
  repo {
    name: "wri-gppd"
    type: "user"
  }
  name: "master"
}
]


In [8]:
s3_client = boto3.client('s3',
                         aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
                         aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],)

with client.commit("essd", "master") as commit:
    for o in bucket.objects.filter(Prefix='ESSD/').all():
        (_, path) = o.key.split('/', 1)
        if not path:
            # Skip ESSD directory all by itself
            continue
        bytes_buffer = io.BytesIO()
        s3_client.download_fileobj(Bucket=bucket.name, Key=o.key, Fileobj=bytes_buffer)
        client.put_file_bytes(commit, "/ESSD/" + path, bytes_buffer.getvalue())

In [9]:
# Check the file commit
print(list(client.list_file(("essd","master"), "/ESSD/")))

[file {
  commit {
    branch {
      repo {
        name: "essd"
        type: "user"
      }
      name: "master"
    }
    id: "867e73d0fe764f2791204b7b1302d6ff"
  }
  path: "/ESSD/essd-13-5213-2021-supplement.pdf"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1660385399
  nanos: 278217000
}
size_bytes: 1108915
hash: "Bw\332P\022\003\014B\364\357e\317\032\'\016\350\353\323\362(\314\212\376\007\272\215\361|\272X\257\377"
, file {
  commit {
    branch {
      repo {
        name: "essd"
        type: "user"
      }
      name: "master"
    }
    id: "867e73d0fe764f2791204b7b1302d6ff"
  }
  path: "/ESSD/essd-13-5213-2021.pdf"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1660385399
  nanos: 278217000
}
size_bytes: 7243436
hash: "3\000\020\314\237\"\321R\202#\006!\031\241ll\377\340\207\350\273\327\034\206s\254\354\002\013r\354\246"
, file {
  commit {
    branch {
      repo {
        name: "essd"
        type: "user"
      }
      name: "master"
    }
    i